In [ ]:
!rm -rf TraindatasetforSSD

In [ ]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download ahemateja19bec1025/trafficsignlocalizationdetectionssdannotated

100% 2.50G/2.50G [01:12<00:00, 46.6MB/s]
100% 2.50G/2.50G [01:12<00:00, 36.9MB/s]


In [ ]:
!unzip \*.zip  && rm *.zip

Показано результат, скорочений до останніх рядків (5000).
  inflating: TraindatasetforSSD/00556(1)B1.jpg  
  inflating: TraindatasetforSSD/00556(1)B1.xml  
  inflating: TraindatasetforSSD/00556(1)B2.jpg  
  inflating: TraindatasetforSSD/00556(1)B2.xml  
  inflating: TraindatasetforSSD/00556(1)B3.jpg  
  inflating: TraindatasetforSSD/00556(1)B3.xml  
  inflating: TraindatasetforSSD/00556(1)B4.jpg  
  inflating: TraindatasetforSSD/00556(1)B4.xml  
  inflating: TraindatasetforSSD/00556B1.jpg  
  inflating: TraindatasetforSSD/00556B1.xml  
  inflating: TraindatasetforSSD/00556B2.jpg  
  inflating: TraindatasetforSSD/00556B2.xml  
  inflating: TraindatasetforSSD/00556B3.jpg  
  inflating: TraindatasetforSSD/00556B3.xml  
  inflating: TraindatasetforSSD/00556B4.jpg  
  inflating: TraindatasetforSSD/00556B4.xml  
  inflating: TraindatasetforSSD/00557(1)B1.jpg  
  inflating: TraindatasetforSSD/00557(1)B1.xml  
  inflating: TraindatasetforSSD/00557(1)B2.jpg  
  inflating: TraindatasetforSSD/005

In [ ]:
!ls 

kaggle.json  sample_data  TraindatasetforSSD


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import hashlib
import imageio
from hashlib import md5
import scipy
import xml.etree.ElementTree as et 
import seaborn as sns
from PIL import Image
from google.colab import files
from tensorflow.keras import models, layers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
def convert_to_jpg(path_file):
  for file in glob.glob(path_file):
    img = cv2.imread(file)
    cv2.imwrite(os.path.splitext(file)[0] + '.jpg', img, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
    os.remove(file)

In [ ]:
convert_to_jpg("/content/TraindatasetforSSD/*.png")
convert_to_jpg("/content/TraindatasetforSSD/*.JPG")

In [ ]:
os.getcwd()

'/content'

In [ ]:
os.chdir(r'/content/TraindatasetforSSD')
os.getcwd()

'/content/TraindatasetforSSD'

In [ ]:
files_list = os.listdir()
print(len(files_list))
print(files_list[0])

7052
000582B1.jpg


In [ ]:
def check_duplicates():
  duplicates = []
  hash_keys = dict()
  for index, filename in  enumerate(os.listdir('.')):  #listdir('.') = current directory
      if os.path.isfile(filename) and os.path.splitext(filename)[1] == ".jpg":
          with open(filename, 'rb') as f:
              filehash = hashlib.md5(f.read()).hexdigest()
          if filehash not in hash_keys: 
              hash_keys[filehash] = index
          else:
              duplicates.append(filename)
  return duplicates

In [ ]:
duplicates_list = check_duplicates()
print(len(duplicates_list))
for file_name in duplicates_list:
    os.remove(file_name)
    os.remove(os.path.splitext(file_name)[0] + '.xml')

12606


In [ ]:
files_list = os.listdir()
print(len(files_list))

7052


In [ ]:
path_xml = "/content/TraindatasetforSSD/*.xml"

In [ ]:
df = pd.DataFrame()
for file_name in glob.glob(path_xml):
  xtree = et.parse(file_name)
  xroot = xtree.getroot() 
  df_cols = ["filename", "width", "height", "depth", "name", "xmin", "ymin", "xmax", "ymax"]
  rows = []

  s_filename = xroot.find("filename").text
  for sizes in xroot.findall("size"):
    s_width = sizes.find("width").text
    s_height = sizes.find("height").text
    s_depth = sizes.find("depth").text

  for objects in xroot.findall('object'):
    s_name = objects.find('name').text

    for bndbox in objects.findall('bndbox'):
      s_xmin = bndbox.find('xmin').text
      s_ymin = bndbox.find("ymin").text 
      s_xmax = bndbox.find("xmax").text
      s_ymax = bndbox.find("ymax").text
    if s_name != "Car" and s_name != "human":
      rows.append({"filename": s_filename, "width": s_width, "height": s_height, "depth": s_depth, 
                    "name": s_name, "xmin": s_xmin, "ymin": s_ymin, "xmax": s_xmax, "ymax": s_ymax})
  df = df.append(rows)
df


,filename,width,height,depth,name,xmin,ymin,xmax,ymax
0,00895B3.jpg,1360,800,3,Speed Limit 70,193,475,236,536
1,00895B3.jpg,1360,800,3,Speed Limit 70,964,414,1019,473
0,00280B2.jpg,1360,800,3,Speed Limit 50,1051,389,1088,425
0,001082B1.jpg,1300,956,3,Speed Limit 100,877,124,1044,291
0,001711B4.jpg,640,480,3,No Entry,123,250,162,293
...,...,...,...,...,...,...,...,...,...
0,00141(1)B4.jpg,1360,800,3,Speed Limit 50,857,378,895,416
1,00141(1)B4.jpg,1360,800,3,RedSignal,29,402,44,428
2,00141(1)B4.jpg,1360,800,3,RedSignal,167,406,180,430
3,00141(1)B4.jpg,1360,800,3,RedSignal,255,459,268,485


In [ ]:
df['name'].value_counts()

Stop                               637
Speed Limit 30                     514
Speed Limit 80                     414
Speed Limit 50                     379
Speed Limit 70                     338
Speed Limit 60                     245
Speed Limit 100                    186
Speed Limit 120                    184
No Entry                           184
GreenSignal                        163
Give Way                           145
No Over Taking Trucks              142
Turn Right                         142
RedSignal                          112
Left Sharp Curve                   104
No Over Taking                      98
Right Sharp Curve                   88
Right Curve Ahead                   84
Danger Ahead                        72
Road Work                           68
Pedestrian                          52
Slippery Road                       46
Traffic Signals Ahead               38
Go Straight                         38
Huddle Road                         36
Snow Warning Sign        

In [ ]:
sign_labels = df['name'].unique() # назви дорожніх знаків
print(sign_labels)
print(len(sign_labels))

['Speed Limit 70' 'Speed Limit 50' 'Speed Limit 100' 'No Entry'
 'Speed Limit 80' 'Speed Limit 120' 'RedSignal' 'Speed Limit 30'
 'Turn Left' 'Stop' 'GreenSignal' 'Road Work' 'Right Curve Ahead'
 'Speed Limit 60' 'Go Straight' 'No Over Taking' 'Danger Ahead' 'Give Way'
 'Go Right or Straight' 'Pedestrian' 'Right Sharp Curve'
 'Left Sharp Curve' 'Turn Right' 'Snow Warning Sign'
 'No Over Taking Trucks' 'Left Curve Ahead' 'No Waiting'
 'Traffic Signals Ahead' 'Slippery Road' 'Huddle Road' 'Deer Zone'
 'End of Right Road (Go straight)' 'No Stopping' 'Truck Sign'
 'Go Left or Straight' 'Cycle Zone' 'RoundAbout' 'YellowSignal']
38


In [ ]:
def add_unique_values(arr1, arr2):
  for value in arr1:
    if value not in arr2:
        arr2.append(value)
  return arr2


In [ ]:
train_img = [] 
validate_img= [] 
test_img = []

for sign in sign_labels:
  #print(sign)
  df_1 = df[df["name"] == sign]
  list_images = df_1["filename"].values.tolist()
  train, validate, test = np.split(list_images, [int(len(list_images)*0.8), int(len(list_images)*0.9)])
  train_img = add_unique_values(train, train_img)
  validate_img = add_unique_values(validate, validate_img)
  test_img = add_unique_values(test, test_img)

print(len(train_img), len(validate_img), len(test_img)) 

Speed Limit 70
Speed Limit 50
Speed Limit 100
No Entry
Speed Limit 80
Speed Limit 120
RedSignal
Speed Limit 30
Turn Left
Stop
GreenSignal
Road Work
Right Curve Ahead
Speed Limit 60
Go Straight
No Over Taking
Danger Ahead
Give Way
Go Right or Straight
Pedestrian
Right Sharp Curve
Left Sharp Curve
Turn Right
Snow Warning Sign
No Over Taking Trucks
Left Curve Ahead
No Waiting
Traffic Signals Ahead
Slippery Road
Huddle Road
Deer Zone
End of Right Road (Go straight)
No Stopping
Truck Sign
Go Left or Straight
Cycle Zone
RoundAbout
YellowSignal
2797 374 368
2797 374 368


In [ ]:
def save_image(path, files, path_file):
  os.chdir("/content/drive/MyDrive/data/" + path)
  for filename in files:
    img = cv2.imread(path_file + filename[:-3] + 'jpg')
    cv2.imwrite(filename, img)


In [ ]:
path_image = '/content/TraindatasetforSSD/'
save_image("train", train_img, path_image)
save_image("validate", validate_img, path_image)
save_image("test", test_img, path_image)

In [ ]:
os.chdir("/content/drive/MyDrive/data")
df.to_csv("data.csv", index=False)

------------------------------------------------------
